In [1]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
from selenium import webdriver
import os
from tqdm import tqdm , trange
import time
import datetime
pd.__version__

'1.4.2'

In [2]:
## Dataset variabele
dataset = "\\huurwoningentotaalvoorpowerbi.xlsx"
## Ophalen van de datasets
pwd = os.getcwd()
oud = pd.read_excel(pwd + dataset)
oud.drop(oud[oud['Status'] == 'Inactive'].index, inplace=True)
oud = oud.fillna("")
oud["Status"] = "TBD"
pd.set_option('display.max_columns', None)

In [3]:
oud

,Plek,Wijk,Postcode,Plaatsnaam,Prijs,Oppervlakte,Kamers,Interieur,Link,AangebodenSinds,Beschikbaarheid,Woningtype,Bouwjaar,Parkeren,Updated,Status
0,Nassaustraat,Nassaubuurt,5046 NS,Tilburg,1180,59,3.0,Gestoffeerd,https://www.huurwoningen.nl/huren/tilburg/1079...,3 weken,Per 30-12-2022,Appartement,1957.0,Nee,2022-11-20,TBD
1,Godijn van Dormaalstraat,Het Lage Land,3067 JJ,Rotterdam,1300,90,3.0,Kaal,https://www.huurwoningen.nl/huren/rotterdam/13...,09-11-2022,Per 15-12-2022,Appartement,1966.0,Nee,2022-11-20,TBD
2,Korte Mare,Noordvest,2312 NN,Leiden,745,30,1.0,Gestoffeerd,https://www.huurwoningen.nl/huren/leiden/13259...,19-11-2022,Per 12-12-2022,Appartement,1760.0,Nee,2022-11-20,TBD
3,Langstraat,Centrum,5801 AA,Venray,591,23,1.0,,https://www.huurwoningen.nl/huren/venray/13259...,19-11-2022,Per 01-01-2023,Appartement,1950.0,Nee,2022-11-20,TBD
4,Pascalerf,Besterd,5014 EW,Tilburg,757,65,1.0,Gestoffeerd,https://www.huurwoningen.nl/huren/tilburg/1325...,19-11-2022,Per 01-01-2023,Appartement,,Nee,2022-11-20,TBD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2420,Regattaweg,Oosterhoogebrug,9731 NA,Groningen,1225,119,3.0,,https://www.huurwoningen.nl/huren/groningen/13...,21-11-2022,Per direct,Appartement,2010.0,Nee,2022-11-22,TBD
2421,Adrienne Solserstraat,Filmwijk,1325 PH,Almere,1295,103,4.0,,https://www.huurwoningen.nl/huren/almere/13261...,21-11-2022,Per direct,Huis,,Nee,2022-11-22,TBD
2422,Barnsteenhorst,Haag Burgen en Horsten,2592 ET,Den Haag,1275,70,3.0,Gemeubileerd,https://www.huurwoningen.nl/huren/den-haag/132...,21-11-2022,Per 15-12-2022,Appartement,1959.0,Ja,2022-11-22,TBD
2423,Sterrenhof,Hooch Boulandt,3511 EV,Utrecht,2025,111,5.0,Gestoffeerd,https://www.huurwoningen.nl/huren/utrecht/1326...,21-11-2022,Per 01-02-2023,Huis,,Ja,2022-11-22,TBD


In [4]:
## Huidige datum krijgen (Jaar-Maand-Dag)
now = datetime.datetime.now()
current_time = now.strftime("%Y-%m-%d")
current_time

'2022-11-23'

In [5]:
url = "https://www.huurwoningen.nl/aanbod-huurwoningen/?page=1/" + "?page="

r = requests.get(url)
soup = bs(r.content)
contents = soup.prettify()
# print(contents)
totaal = soup.find(class_="search-list")
# print(totaal.prettify())

In [6]:
pagina = 1
link = []
title=[]
plek=[]
adres=[]
prijs=[]
oppervlakte = []
kamers = []
interieur = []
totpages = soup.select(".pagination__item a")[4].text

for pagina in trange(int(totpages)):
    url = "https://www.huurwoningen.nl/aanbod-huurwoningen/?page=" + str(pagina)

    r = requests.get(url)
    soup = bs(r.content)
    contents = soup.prettify()
    # print(contents)
    totaal = soup.find(class_="search-list")
    # print(totaal.prettify())

    linktitle = soup.select(".listing-search-item__title a")
    subtitle = soup.select(".listing-search-item__sub-title")
    prijstitle = soup.select(".listing-search-item__price")
    info = soup.select(".listing-search-item__features")

    i = 0

    for i in range(len(linktitle)):
        link.append('https://www.huurwoningen.nl' + linktitle[i]['href'])
        title.append(linktitle[i].text.strip())
        plek.append(subtitle[i].text.strip().split(' ',3)[3].replace('(','').replace(')',''))
        adres.append(subtitle[i].text.strip().split(' (')[0])
        prijs.append(prijstitle[i].text.strip().split(' ')[0][2:])
        m2 = info[i].select(".illustrated-features__item--surface-area")
        oppervlakte.append(m2[0].text.split(' ')[0])
        rooms = info[i].select(".illustrated-features__item--number-of-rooms")
        try : 
            kamers.append(rooms[0].text.split(' ')[0])
        except IndexError:
            kamers.append("")
        intr = info[i].select(".illustrated-features__item--interior")
        try :
            interieur.append(intr[0].text)
        except IndexError:
            interieur.append("")
        i = i + 1
    pagina = pagina + 1
    # print(pagina)
    time.sleep(0.3)

100%|██████████| 66/66 [01:12<00:00,  1.10s/it]


In [7]:
prijs2 = []
for i in prijs:
    prijs2.append(i.replace(".",""))

In [8]:
df = pd.DataFrame()
prijs2 = []
for i in prijs:
    prijs2.append(i.replace(".",""))
df["Plek"] = title
df["Wijk"] = plek
df["Postcode"] = adres
df["Plaatsnaam"] = df["Postcode"].str.split(' ',n=2).str[2]
df["Postcode"] = df["Postcode"].str.split(' ').str[0] + " " +  df["Postcode"].str.split(' ').str[1]
df["Prijs"] = prijs2
df["Oppervlakte"] = oppervlakte
df["Kamers"] = kamers
df["Interieur"] = interieur
df["Link"] = link
df.drop_duplicates()
df

,Plek,Wijk,Postcode,Plaatsnaam,Prijs,Oppervlakte,Kamers,Interieur,Link
0,Gerard Doustraat,Ossenkoppelerhoek Midden Noord,7606 GD,Almelo,350,85,2,Kaal,https://www.huurwoningen.nl/huren/almelo/13158...
1,Godijn van Dormaalstraat,Het Lage Land,3067 JJ,Rotterdam,1300,90,3,Kaal,https://www.huurwoningen.nl/huren/rotterdam/13...
2,Bartolomeus Diazstraat,Columbuskwartier,1363 KC,Almere,1650,76,3,Gemeubileerd,https://www.huurwoningen.nl/huren/almere/13266...
3,Bussumsestraat,Haag Rustenburg,2574 JG,Den Haag,1300,57,3,Gestoffeerd,https://www.huurwoningen.nl/huren/den-haag/132...
4,Iepenpad,Aar Ter Aar-Centrum voorheen Aardam,2461 CZ,Ter Aar,850,56,3,,https://www.huurwoningen.nl/huren/ter-aar/1326...
...,...,...,...,...,...,...,...,...,...
2107,Kerkachterweg,Beatrixstraat,3171 GC,Poortugaal,530,80,,,https://www.huurwoningen.nl/huren/poortugaal/1...
2108,Plantage Kerklaan,Weesperbuurt/Plantage,1018 SX,Amsterdam,2000,75,2,Gemeubileerd,https://www.huurwoningen.nl/huren/amsterdam/13...
2109,Hoogstraat,Markt,4331 KR,Middelburg,1200,107,3,Gemeubileerd,https://www.huurwoningen.nl/huren/middelburg/1...
2110,Oranjelaan,Dorp,2712 GD,Zoetermeer,1300,62,3,Gemeubileerd,https://www.huurwoningen.nl/huren/zoetermeer/1...


In [9]:
merged = pd.merge(df, oud, on='Link', indicator=True, how ='outer')
datadif = merged.loc[lambda x : x['_merge'] != 'both']

In [10]:
dfdif1 = datadif.loc[datadif['_merge'] == 'left_only']
dfdif2 = datadif.loc[datadif['_merge'] == 'right_only']
di1 = oud[0:0]
di2 = oud[0:0]


In [11]:
prijsy = []
for i in dfdif2['Prijs_y']:
    prijsy.append(str(i).split('.0')[0])
dfdif2['Prijs_y'] = prijsy
oppervlaktey = []
for i in dfdif2['Oppervlakte_y']:
    oppervlaktey.append(str(i).split('.0')[0])
dfdif2['Oppervlakte_y'] = oppervlaktey
kamersy = []
for i in dfdif2['Kamers_y']:
    kamersy.append(str(i).split('.0')[0])
dfdif2['Kamers_y'] = kamersy
bouwjaary = []
for i in dfdif2['Bouwjaar']:
    bouwjaary.append(str(i).split('.0')[0])
dfdif2['Bouwjaar'] = bouwjaary

C:\Users\Roald\AppData\Local\Temp\ipykernel_21516\87617565.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfdif2['Prijs_y'] = prijsy
C:\Users\Roald\AppData\Local\Temp\ipykernel_21516\87617565.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfdif2['Oppervlakte_y'] = oppervlaktey
C:\Users\Roald\AppData\Local\Temp\ipykernel_21516\87617565.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the c

In [12]:
di1["Plek"] = dfdif1["Plek_x"]
di1["Wijk"] = dfdif1["Wijk_x"]
di1["Postcode"] = dfdif1["Postcode_x"]
di1["Plaatsnaam"] = dfdif1["Plaatsnaam_x"]
di1["Prijs"] = dfdif1["Prijs_x"]
di1["Oppervlakte"] = dfdif1["Oppervlakte_x"]
di1["Kamers"] = dfdif1["Kamers_x"]
di1["Interieur"] = dfdif1["Interieur_x"]
di1["Link"] = dfdif1["Link"]
di1["Updated"] = current_time
di1["Status"] = 'Active'
di1 = di1.reset_index()

di2["Plek"] = dfdif2["Plek_y"]
di2["Wijk"] = dfdif2["Wijk_y"]
di2["Postcode"] = dfdif2["Postcode_y"]
di2["Plaatsnaam"] = dfdif2["Plaatsnaam_y"]
di2["Prijs"] = dfdif2["Prijs_y"]
di2["Oppervlakte"] = dfdif2["Oppervlakte_y"]
di2["Kamers"] = dfdif2["Kamers_y"]
di2["Interieur"] = dfdif2["Interieur_y"]
di2["Link"] = dfdif2["Link"]
di2["AangebodenSinds"] = dfdif2["AangebodenSinds"]
di2["Beschikbaarheid"] = dfdif2["Beschikbaarheid"]
di2["Woningtype"] = dfdif2["Woningtype"]
di2["Bouwjaar"] = dfdif2["Bouwjaar"]
di2["Parkeren"] = dfdif2["Parkeren"]
di2["Updated"] = dfdif2["Updated"]
di2["Status"] = 'Inactive'
di2 = di2.reset_index()
di2 = di2.drop(columns='index')

C:\Users\Roald\AppData\Local\Temp\ipykernel_21516\371274350.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  di1["Plek"] = dfdif1["Plek_x"]
C:\Users\Roald\AppData\Local\Temp\ipykernel_21516\371274350.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  di1["Wijk"] = dfdif1["Wijk_x"]
C:\Users\Roald\AppData\Local\Temp\ipykernel_21516\371274350.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cav

In [13]:
sinds = []
beschikbaar = []
woningtype = []
jaarbouw = []
parkeer = []

for i in trange(len(di1["Link"])):
    url = str(di1["Link"][i])

    r = requests.get(url)
    soup = bs(r.content)
    contents = soup.prettify()
    # print(contents)
    totaal = soup.find(class_="page__row page__row--features")
    # print(totaal.prettify())

    aangebodensinds = soup.select(".listing-features__description--offered_since")
    beschikbaarheid = soup.select(".listing-features__description--acceptance")
    typewoning = soup.select(".listing-features__description--dwelling_type")
    bouwjaar = soup.select(".listing-features__description--construction_period")
    parkeren = soup.select(".listing-features__description--available")

    try: 
        sinds.append(aangebodensinds[0].text.strip())
    except IndexError:
        sinds.append("")
    try:
        beschikbaar.append(beschikbaarheid[0].text.strip())
    except IndexError:
        beschikbaar.append("")
    try:
        woningtype.append(typewoning[0].text.strip().split()[0])
    except IndexError:
        woningtype.append("")
    try: 
        jaarbouw.append(bouwjaar[0].text.strip())
    except IndexError:
        jaarbouw.append("")
    try: 
        parkeer.append(parkeren[0].text.strip())
    except:
        parkeer.append("")
    i = i + 1
    time.sleep(0.3)
    

100%|██████████| 252/252 [03:10<00:00,  1.33it/s]


In [14]:
di1["AangebodenSinds"] = sinds
di1["Beschikbaarheid"] = beschikbaar
di1["Woningtype"] = woningtype
di1["Bouwjaar"] = jaarbouw
di1["Parkeren"] = parkeer
di1.drop_duplicates()
di1 = di1.drop(columns='index')

In [15]:
inner1 = pd.concat([oud, di1])
inner1.loc[inner1.duplicated(keep=False), :]

,Plek,Wijk,Postcode,Plaatsnaam,Prijs,Oppervlakte,Kamers,Interieur,Link,AangebodenSinds,Beschikbaarheid,Woningtype,Bouwjaar,Parkeren,Updated,Status
0,Bartolomeus Diazstraat,Columbuskwartier,1363 KC,Almere,1650,76,3,Gemeubileerd,https://www.huurwoningen.nl/huren/almere/13266...,23-11-2022,Per 12-12-2022,Huis,2008,Ja,2022-11-23,Active
1,Bartolomeus Diazstraat,Columbuskwartier,1363 KC,Almere,1650,76,3,Gemeubileerd,https://www.huurwoningen.nl/huren/almere/13266...,23-11-2022,Per 12-12-2022,Huis,2008,Ja,2022-11-23,Active
2,Bussumsestraat,Haag Rustenburg,2574 JG,Den Haag,1300,57,3,Gestoffeerd,https://www.huurwoningen.nl/huren/den-haag/132...,23-11-2022,Per direct,Appartement,1926,Nee,2022-11-23,Active
3,Bussumsestraat,Haag Rustenburg,2574 JG,Den Haag,1300,57,3,Gestoffeerd,https://www.huurwoningen.nl/huren/den-haag/132...,23-11-2022,Per direct,Appartement,1926,Nee,2022-11-23,Active
4,Iepenpad,Aar Ter Aar-Centrum voorheen Aardam,2461 CZ,Ter Aar,850,56,3,,https://www.huurwoningen.nl/huren/ter-aar/1326...,23-11-2022,Per 01-12-2022,Appartement,1984,Nee,2022-11-23,Active
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206,Van der Helmstraat,Het Lage Land,3067 HM,Rotterdam,1275,85,3,Gemeubileerd,https://www.huurwoningen.nl/huren/rotterdam/12...,23-11-2022,Per 15-12-2022,Appartement,1966,Ja,2022-11-23,Active
207,Van der Helmstraat,Het Lage Land,3067 HM,Rotterdam,1275,85,3,Gemeubileerd,https://www.huurwoningen.nl/huren/rotterdam/12...,23-11-2022,Per 15-12-2022,Appartement,1966,Ja,2022-11-23,Active
208,Van der Helmstraat,Het Lage Land,3067 HM,Rotterdam,1275,85,3,Gemeubileerd,https://www.huurwoningen.nl/huren/rotterdam/12...,23-11-2022,Per 15-12-2022,Appartement,1966,Ja,2022-11-23,Active
209,Van der Helmstraat,Het Lage Land,3067 HM,Rotterdam,1275,85,3,Gemeubileerd,https://www.huurwoningen.nl/huren/rotterdam/12...,23-11-2022,Per 15-12-2022,Appartement,1966,Ja,2022-11-23,Active


In [16]:
inner1.drop_duplicates(keep='first').shape
inner2 = pd.concat([inner1, di2])
inner2 = inner2.drop_duplicates(keep='last')
inner2["Status"].value_counts()

TBD         2425
Inactive     681
Active       216
Name: Status, dtype: int64

In [17]:
inner2["Status"] = inner2["Status"].replace("Active","New")
inner2["Status"] = inner2["Status"].replace("TBD","Active")
inner2["Status"].value_counts()

Active      2425
Inactive     681
New          216
Name: Status, dtype: int64

In [18]:
bouwjaary = []
for i in inner2['Bouwjaar']:
    bouwjaary.append(str(i).split('.0')[0])
inner2['Bouwjaar'] = bouwjaary

In [19]:
oud = oud.fillna("")

In [20]:
pwd = os.getcwd()
name = "huurwoningentotaal"
inner2.to_excel(pwd + "\\" + name + current_time + ".xlsx", index=False)
inner2.to_excel(pwd + "\\" + name + 'voorpowerbi' + ".xlsx", index=False)